# Aperçu de méthodes statistiques dans l’attribution de la paternité ou de la maternité d’une œuvre littéraire

Avant l’avènement des techniques d’apprentissage automatique, les tâches d’attribution de la paternité ou de la maternité d’une œuvre littéraire se résolvaient grâce à des méthodes statistiques qui ont encore cours aujourd’hui au point d’alimenter parfois les premières.

L’idée générale est de comparer la fréquence de traits linguistiques (mots, mots outils, *n*-grammes, structures syntaxiques…) entre un ou plusieurs corpus aux auteurs avérés avec un texte anonyme ou à l’attribution contestée. Les tâches concernées par ces méthodes peuvent aller de la simple vérification d’une attribution à la classification (décider s’il s’agit d’un roman policier ou de science-fiction) en passant par l’arbitrage entre deux ou plusieurs auteur·rices.

Après avoir abordé rapidement quelques notions fondamentales sur les tests statistiques, nous appliquerons certains d’entre eux à des cas précis.

## Des hypothèses statistiques

Communément, un test statistique sert à vérifier qu’une hypothèse est vraie : c’est *l’hypothèse nulle*, notée $H_0$. Dans le cas contraire, *l’hypothèse alternative*, notée $H_1$, est retenue car considérée comme statistiquement significative. Pour juger de la significativité du test, le résultat de l’expérience est confronté à une loi de probabilité avec un seuil de risque $\alpha$ généralement établi à 0,05 (5 % de risque d’erreur). Si la probabilité d’obtenir une valeur aussi extrême que celle observée, la valeur-p (ou *p-value* en anglais), est inférieure à ce seuil, de fortes présomptions pèsent contre l’hypothèse nulle que l’on vient de tester. À l’inverse, si la valeur-p est supérieure à ce seuil, on conclut simplement à l’échec du test, sans ne rien dire des hypothèses formulées.

## Les courbes caractéristiques de composition

En 1887, Thomas Corwin Mendenhall publie un article dans *Science*, "[The Characteristic Curves of Composition](https://www.science.org/doi/10.1126/science.ns-9.214S.237)", dans lequel il avance que non seulement le style d’un auteur est régulier tout au long d’une de ses œuvres mais qu’il se retrouve en plus dans l’ensemble de ses ouvrages. Afin de le prouver, il retient comme mesure la longueur des mots dont la distribution révèlerait une signature caractéristique.

Aujourd’hui dépassée par l’évolution de l’informatique, la méthode de Mendenhall vaut surtout pour l’anecdote historique.

## Le test du $\chi^2$ d’indépendance

### Principe du test d’indépendance

L’une des applications connues du test du $\chi^2$ intervient dans l’estimation de la liaison entre deux variables qualitatives : soit elles sont indépendantes ($H_0$) soit elles covarient de manière statistiquement significative ($H_1$).

Pour parvenir au résultat, la méthode consiste à établir, pour les variables ciblées, la distance entre les valeurs empiriques ($O_{ij}$) et les valeurs théoriques normalement attendues ($E_{ij}$) si elles étaient indépendantes, selon la formule :

$$
\chi^2 = \sum_{i,j} \frac{(O_{ij} - E_{ij})^2}{E_{ij}}
$$

La somme obtenue est alors comparée à [la loi du $\chi^2$](https://fr.wikipedia.org/wiki/Loi_du_%CF%87%C2%B2#Table_de_valeurs_des_quantiles) avec $k$ degrés de liberté pour un risque d’erreur de 5 %, sachant que les degrés de liberté s’obtiennent en effectuant le produit entre le nombre de lignes et le nombre de colonnes selon la formule :

$$
k = (I – 1)(J – 1)
$$

### Comment survivre au naufrage du *Titanic*

Émettons l’hypothèse qu’être riche nous aurait fourni plus de chances pour survivre au naufrage du *Titanic*. Mais comment s’en assurer ? Mobilisons [un jeu de données](./data/titanic.csv) qui fournit quelques informations sur les passagers du fameux paquebot :

In [ ]:
import pandas as pd

df = pd.read_csv('./data/titanic.csv')

display(df.head())

Retenons les variables `Pclass` et `Survived`, la première indiquant la classe de transport (1e, 2e ou 3e classe) et la seconde nous apprenant si la personne a survécu ou non (1 ou 0) :

In [ ]:
data = df[["Survived", "Pclass"]]

display(data.head())

Créons maintenant une table de contingence afin de savoir combien de passagers et passagères ont péri ou survécu en fonction de leurs conditions d’hébergement à bord :

In [ ]:
contingency = pd.crosstab(df.Pclass, df.Survived)

display(contingency)

Dans ce tableau, nous lisons par exemple que 80 personnes de la première classe ont péri contre 372 de la seconde classe. À titre informatif, notons qu’il est possible d’obtenir les sommes marginales :

In [ ]:
pd.crosstab(df.Pclass, df.Survived, margins=True)

De là, nous devons calculer les valeurs théoriques attendues ($E_{ij}$) en cas d’indépendance des variables :

$$
E_{ij} = \frac{ \sum_{j=1}^{J}O_{ij} \cdot \sum_{i=1}^{I}O_{ij}}{N}
$$

|Pclass|0|1|
|:-:|:-:|:-|
|1|(216 * 549) / 891 = **133.09**|(216 * 342) / 891 = **82.91**|
|2|(184 * 549) / 891 = **113.37**|(184 * 342) / 891 = **70.63**|
|3|(491 * 549) / 891 = **302.54**|(491 * 342) / 891 = **188.46**|

La matrice $E_{ij}$ peut s’obtenir directement avec Python :

In [ ]:
N = contingency.sum().sum()
Oi = contingency.sum(axis=1).values
Oj = contingency.sum(axis=0).values

# reshape Oi
Oi = Oi.reshape(contingency.shape[0], 1)

Eij = (Oi * Oj) / N

display(Eij)

Il ne reste plus qu’à calculer la valeur du $\chi^2$ :

In [ ]:
chi_squared = (contingency.values - Eij) ** 2 / Eij

display(chi_squared.sum())

#### Interprétation avec la table de distribution du $\chi^2$

La [loi du $\chi^2$](https://fr.wikipedia.org/wiki/Loi_du_%CF%87%C2%B2#Table_de_valeurs_des_quantiles) fournit des valeurs critiques pour chaque niveau de seuil $\alpha$ avec $k$ degrés de liberté. Dans notre exemple, nous avons retenu un seuil de risque de 0,05 et le nombre de degrés de liberté est de $(3 - 1)(2 - 1) = 2$. La valeur critique donnée par la table est de 5,99. Notre résultat lui étant bien supérieur, nous pouvons rejeter $H_0$.

#### Interprétation avec la valeur-p

Plus communément, l’interprétation d’un test statistique se fait avec la valeur-p qui dans notre exemple exprime la probabilité d’obtenir une valeur aussi extrême que celle observée. Si la valeur-p est inférieure au seuil de 0,05, nous pouvons rejeter l’hypothèse nulle.

La méthode `chi2.sf()` de *Scipy* permet de calculer la valeur-p à partir d’un $\chi^2$ et du nombre de degrés de libertés :

In [ ]:
from scipy.stats import chi2

display(chi2.sf(chi_squared.sum(), 2))

À noter l’existence d’une fonction `chi2_contingency()` qui ressort toutes les mesures désirées à partir d’un tableau de contingence sans les sommes marginales :

In [ ]:
from scipy.stats import chi2_contingency

chi2, pvalue, degrees, expected = chi2_contingency(contingency)

print(
    f"Valeur du khi-deux : {chi2:.4f}",
    f"Probabilité du khi-deux (valeur-p) : {pvalue:.4f}",
    sep="\n"
)

### Test de la distance entre deux vocabulaires

Formulons une nouvelle hypothèse selon laquelle un texte anonyme n’est pas de la main d’un auteur particulier. Pour la tester, nous envisageons de calculer la distance entre la fréquence des occurrences des mots dans le texte anonyme et celle observée dans le corpus de l’auteur. Si à la fin du test la valeur-p est inférieure au seuil de 5 %, nous présumons que notre hypothèse est fausse et retenons l’hypothèse alternative.

La méthode consiste à :

1. obtenir les vocabulaires du texte anonyme et du corpus de l’auteur ;
2. les combiner afin d’obtenir un vocabulaire complet ;
3. sélectionner les $n$ mots les plus fréquents du corpus combiné ;
4. calculer les valeurs théoriques ($E_{ij}$) dans chacun des deux corpus ;
5. calculer la distance avec les valeurs empiriques.

Avant de commencer, chargeons toutes les bibliothèques logicielles nécessaires ainsi que les textes à comparer :

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from scipy.stats import chi2

tokenizer = RegexpTokenizer('\w+')

corpus = [
    "Le petit chat boit du lait.",
    "Le petit chien boit de l’eau.",
    "La vache boit de l’eau mais ne boit pas de lait."
]

anonym = "Le petit oiseau boit de l’eau."

#### Étape 1 : obtenir les vocabulaires

La réalisation de cette étape consiste à appliquer toutes les opérations mises en jeu dans une tâche de sac de mots. Nous nous contentons ici d’une banale tokenisation afin d’obtenir au final deux matrices d’occurrences.

##### La matrice des occurrences du corpus

La segmentation est effectuée pour chaque texte de la variable `corpus` en filtrant sur les mots de deux caractères et moins :

In [ ]:
words_corpus = [
    word.lower()
    for sent in corpus
    for word in tokenizer.tokenize(sent)
    if len(word) > 2
]

Grâce à un objet de type `Counter()`, il est maintenant aisé d’obtenir les fréquences d’apparition des mots dans le corpus :

In [ ]:
occurrences_corpus = Counter(words_corpus)

##### La matrice d’occurrences du texte anonyme

Répétons l’opération pour le texte anonyme :

In [ ]:
occurrences_anonym = Counter([
    word.lower()
    for word in tokenizer.tokenize(anonym)
    if len(word) > 2
])

#### Étape 2 : combiner les deux vocabulaires

Une opération triviale qui consiste à fusionner les deux objets de type `Counter` en un troisième :

In [ ]:
occurrences_all = occurrences_corpus + occurrences_anonym

Regardons un aperçu du résultat dans un *data frame* :

In [ ]:
pd.DataFrame(
    data=occurrences_all.values(),
    index=occurrences_all.keys(),
    columns=["All"]
)

Les *data frames* sont en vérité des structures très pratiques pour combiner et afficher des données. Leur compatibilité avec les structures natives de Python et des biblbiothèques de référence comme *Numpy* en font un incontournable de l’analyse de données.

Affichons une représentation de la distribution de notre vocabulaire parmi les deux documents :

In [ ]:
df = pd.DataFrame(
    data=[occurrences_corpus, occurrences_anonym],
    index=["A", "X"]
).fillna(0)

display(df)

#### Étape 3 : sélectionner les mots les plus fréquents

Un corpus étant généralement constitué d’une très grande quantité, il conviendra dans la réalité de sélectionner les *n* mots les plus fréquents du vocabulaire combiné. Pour les besoins de l’exercice, nous sélectionnerons les 20 mots les plus fréquents :

In [ ]:
most_common = occurrences_all.most_common(20)

# remove occurrences to obtain a simple list
most_common = list(map(lambda x: x[0], most_common))

Et filtrons les objets `Counter` afin de ne retenir que les clés désirées :

In [ ]:
occurrences_all = Counter({ key: value for key, value in occurrences_all.items() if key in most_common })
occurrences_corpus = Counter({ key: value for key, value in occurrences_corpus.items() if key in most_common })
occurrences_anonym = Counter({ key: value for key, value in occurrences_anonym.items() if key in most_common })

L’opération est en fait plus simple lorsqu’elle est appliquée directement au *data frame* :

In [ ]:
df = df[df.columns.intersection(most_common)]

#### Étape 4 : calculer les valeurs théoriques

Le tableau se présentant avec des effectifs différents répartis dans des catégories similaires, nous pouvons appliquer le test du $\chi^2$ d’indépendance et calculer la distribution marginale afin d’obtenir les valeurs théoriques ($E_{ij}$) :

In [ ]:
N = df.sum().sum()
Oi = df.sum(axis=1).values
Oj = df.sum(axis=0).values

# reshape Oi
Oi = Oi.reshape(df.shape[0], 1)

Eij = (Oi * Oj) / N

display(Eij)

#### Étape 5 : calculer la distance entre les textes

Il ne reste plus qu’à appliquer la formule pour trouver la valeur du $\chi^2$ :

In [ ]:
chi_squared = (df.values - Eij) ** 2 / Eij

display(chi_squared.sum())

Et pour les degrés de liberté :

In [ ]:
k = (Oi.shape[0] - 1) * (Oj.shape[0] - 1)

#### Interpréter les résultats

La valeur critique fournie par la loi du $\chi^2$ pour un risque $\alpha$ de 0,05 avec 9 degrés de liberté est de 16,92. La valeur que nous avons calculée étant inférieure, nous ne pouvons pas rejeter l’hypothèse nulle et ne pouvons donc pas affirmer que les deux textes sont du même auteur.

Le calcul de la valeur-p renvoie à la même conclusion :

In [ ]:
pvalue = chi2.sf(chi_squared.sum(), k)

print(
    f"Valeur-p : {pvalue}",
    "La valeur-p étant supérieure au seuil de 0,05, nous ne pouvons pas rejeter l’hypothèse nulle.",
    sep="\n"
)

### Du malaise dans la statistique textuelle

Le résultat que nous avons précédemment obtenu provient d’un test statistique durement éprouvé, conçu en 1900 par un mathématicien britannique, Karl Pearson, et diffusé grâce à un article paru dans [*The London, Edinburgh, and Dublin Philosophical Magazine and Journal of Science*](https://www.tandfonline.com/doi/abs/10.1080/14786440009463897). Comme nous l’avons vu, l’application de la formule du $\chi^2$ repose sur des données quantitatives afin de fournir une valeur censée vérifier une hypothèse. Que ce soit dans le premier exemple sur nos chances de survie à bord du *Titanic* ou dans le second sur l’indépendance entre un corpus et un texte anonyme, cette première exigence a été remplie. Pourquoi *première* ? Parce que le recours au test du $\chi^2$ supppose davantage de prérequis, parmi lesquels :

- des effectifs de 5 minimum ;
- des variables indépendantes ;
- des données aléatoires ;
- …

Le dernier critère que nous citons est moins anodin qu’il n’y paraît : une variable aléatoire suppose des données aléatoirement distribuées. Autrement dit, la présence des mots dans le corpus ne doit pas être explicable par leur entourage, ce qui est contradictoire avec l’idée même de langage. Rien de plus naturel en effet à trouver en présence du substantif *chat* le déterminant *le*. Pour cette raison, en 2001, Adam Kilgarriff propose une variante du test du $\chi^2$ d’indépendance qui se débarrasse de l’hypothèse nulle.

#### Le $\chi^2$ de Kilgarriff

Dans son article ["Comparing Corpora"](https://doi.org/10.1075/ijcl.6.1.05kil) paru en 2001 dans le *International Journal of Corpus Linguistics*, Adam Kilgarriff met en lumière les problèmes soulevés par l’application de tests statistiques à la matière textuelle. Non seulement la distribution des mots dans un corpus n’est pas le fruit du hasard, mais cette influence réciproque qu’ils exercent entre eux rend caduque le recours à certaines hypothèses. Franck Owen et Ron Jones font remarquer en 1977 dans [*Statistics*](https://annas-archive.org/md5/30509e973a8e1198e80c20bf1694c8da) qu’à partir d’une certaine taille d’échantillon toutes les hypothèses nulles dans le cadre d’un test du $\chi^2$ seraient rejetées.

Kilgarriff propose une méthode d’évaluation du $\chi^2$ reposant sur une constitution rigoureuse des corpus à comparer, trop souvent oubliée par la littérature scientifique qui se contente de ne conserver de son article que la formule. Les expériences menées par Kilgarriff, qu’elles portent sur le $\chi^2$ ou sur d’autres tests statistiques, impliquaient au préalable de constituer un ensemble de *Known-Similarity Corpora* (KSC) à partir de deux textes raisonnablement différents. Dans un ensemble KSC, le premier corpus contient 100 % du premier texte, le second 90 % du premier et 10 % du second etc., ce qui permet d’émettre des affirmations comme : le premier corpus est plus similaire au second qu’au troisième, ou bien le second corpus est plus similaire au troisième qu’il ne l’est au quatrième et ainsi de suite. À partir de là, une bonne mesure serait celle qui validerait toutes les affirmations.

Une fois l’ensemble KSC constitué, la méthode de Kilgarriff propose de suivre les étapes suivantes :

1. diviser un corpus en tranches (500 mots) ;
2. créer deux sous-corpus en affectant aléatoirement la moitié de chaque ;
3. mesurer leur similarité ;
4. reproduire les étapes 2 et 3 un certain nombre de fois ;
5. calculer la moyenne et l’écart-type de toutes les itérations.

## Tests basés sur les rangs des mots

Il existe des tests statistiques qui se prêtent mieux à l’étude d’un texte que d’autres. Dans le cas du $\chi^2$, nous avons notamment relevé qu’il impliquait une distribution aléatoire des mots alors que toute production d’un énoncé un tant soit peu compréhensible suppose le respect d’un modèle de langue.

Parmi ces tests, dits non paramétriques, certains prennent pour données de départ le rang d’un item dans un ensemble ordonné plutôt que sa valeur. Il s’agit par exemple du test U de Mann-Whitney ou encore du coefficient de corrélation de rang de Spearman.

### Le test $U$ d’indépendance de Mann-Whitney

#### Dans un ensemble d’échantillons de moins de vingt paires

Sous l’hypothèse nulle, ce test pose la similitude entre deux groupes d’une population. En analyse de textes, il s’agira de tester deux corpus en comparant des échantillons de tailles similaires. On calculera la fréquence d’un mot *w* dans chacun des échantillons et ordonnerons les résultats. Prenons pour exemple huit échantillons de 5000 mots issus de deux textes A et B et regardons la distribution des occurrences du mot *chat* :

||||||||||Total|
|-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|Fréquence|4|12|12|15|21|27|31|75|–|
|Échantillon|B3|B1|A2|B4|A1|A3|A4|B2|–|
|Rang (A)|–|–|2,5|–|5|6|7|–|20,5|
|Rang (B)|1|2,5|–|4|–|–|–|8|15,5|

**Remarque :** en cas d’égalité de rang, on les additionne pour attribuer la moyenne à toutes les valeurs concernées par l’égalité.

On note ensuite les valeurs de $U_A$ et $U_B$ :

$$
\begin{align}
U_A &= n_1 \cdot n_2 + \frac{n_1 \cdot (n_1 + 1)}{2} - R_1\\
&= 4 \cdot 4 + \frac{4 \cdot (4 + 1)}{2} - 20,5\\
&= 16 + \frac{20}{2} - 20,5\\
&= 16 + 10 - 20,5\\
&= 5,5\\
U_B &= n_1 \cdot n_2 + \frac{n_2 \cdot (n_2 + 1)}{2} - R_2\\
&= 16 + 10 - 15,5\\
&= 10,5
\end{align}
$$

Pour ne retenir que le plus petit score – à savoir ici 5.5 – comme valeur de $U$. Si le nombre d’échantillons n’excède pas une vingtaine de paires, alors le résultat du test est lisible directement dans [une table de Mann-Whitney](images/mann-whitney.png). Dans notre exemple, la valeur critique au seuil $\alpha$ de 0.05 est de 0. Comme $5 > 0$, nous ne pouvons rejeter l’hypothèse nulle et devons accepter qu’il n'y a pas de différence notable entre les deux ensembles d’échantillons.

#### Dans un grand ensemble d’échantillons

Si les ensembles que nous comparons sont constitués de plus d’une vingtaine d’échantillons, nous aurons alors besoin de calculer une valeur-z :

$$
z = \frac{U - \mu U}{\sigma U}
$$

Où $\mu U$ représente la valeur attendue de $U$ :

$$
\mu U = \frac{n_1 \cdot n_2}{2}
$$

et $\sigma U$ son écart-type :

$$
\sigma U = \sqrt{\frac{n_1 \cdot n_2 \cdot (n_1 + n_2 + 1)}{12}}
$$

Considérons un ensemble d’une cinquantaine d’échantillons caractérisé par :

$$
\begin{align}
    n_1 &= 25\\
    n_2 &= 25\\
    R_1 &= 636\\
    R_2 &= 639\\
    U_1 &= 25 \cdot 25 + \frac{25 \cdot (25 + 1)}{2} - 636 = 314\\
    U_2 &= 25 \cdot 25 + \frac{25 \cdot (25 + 1)}{2} - 639 = 311
\end{align}
$$

Nous obtenons une espérance de U de :

$$
\mu U = 25 \cdot 25 \div 2 = 312,5
$$

Et un écart-type de :

$$
\begin{align}
    \sigma U &= \sqrt{\frac{25 \cdot 25 \cdot (25 + 25 + 1)}{12}}\\
    &= \sqrt{\frac{625 \cdot 51}{12}}\\
    &= \sqrt{\frac{31875}{12}}\\
    &= 51,5388
\end{align}
$$

Maintenant que nous avons la moyenne et l’écart-type attendus, nous pouvons calculer la valeur-z :

$$
\begin{align}
    z &= \frac{311 - 312,5}{51,5388}\\
    &= \frac{-1.5}{51,5388}\\
    &\approx -0,0291
\end{align}
$$

La valeur-z permet de se repérer dans la table de la loi normale centrée réduite, d’où nous pouvons déduire que, pour une valeur-z approchant de -0,0291, la valeur-p est de 0,977, ce qui ne nous permet pas de rejeter l’hypothèse nulle selon laquelle il n’y a pas de différence entre les deux classes.

#### Ampleur de l’effet

Dans l’hypothèse où nous aurions mis en lumière une différence significative entre les deux classes, nous aurions pu également mesurer l’ampleur de l’effet selon la formule :

$$
r = \frac{\lvert z \rvert}{\sqrt{n}}
$$

#### Effectuer le test avec Python

La bibliothèque *Scipy* offre une méthode `.mannwhitneyu()` pour passer le test. Reprenons le premier exemple avec la distribution du mot *chat* dans deux corpus divisés en huit échantillons :

In [ ]:
from scipy.stats import mannwhitneyu

# occurrences of word 'chat' in two corpora,
# divided into eight samples
A = [12, 21, 27, 31]
B = [4, 12, 15, 75]

# method returns the value of U for the first sample,
# test is two-sided by default
U1, p = mannwhitneyu(A, B, method="exact", alternative='two-sided')

# U of the second sample can be easily computed
U2 = len(A) * len(B) - U1

# value of U
U = min(U1, U2)

print(f"La valeur U de Mann-Whitney est de {U} avec une valeur-p de {p}")

La probabilité de rencontrer une distribution au moins aussi extrême du mot *chat* étant supérieure à 5 %, les résultats du test ne sont pas significatifs et nous ne pouvons pas rejeter l’hypothèse nulle d’absence de différence entre les corpus.

### Le coefficient de corrélation de rang de Spearman

Une autre mesure basée sur le rang des mots et qui permet d’évaluer la similarité de deux corpus est la corrélation de rang de Spearman :

$$
\rho = 1 - \frac{6 \sum d^2_i}{n \cdot (n^2 -1)}
$$

Elle suppose de comparer des objets de dimensions équivalentes et sans égalité de rang où tous les mots d’un corpus seront dans le second corpus. Considérons l’exemple suivant où nous souhaitons étudier deux corpus A et B dont les cinq mots les plus fréquents sont :

$$
\begin{align}
    A &= (\text{chat}, 12), (\text{eau}, 9), (\text{lait}, 9), (\text{chien}, 8), (\text{boire}, 6)\\
    B &= (\text{chat}, 85), (\text{lait}, 42), (\text{boire}, 38), (\text{chien}, 22), (\text{eau}, 17)
\end{align}
$$

Nous obtenons le tableau suivant :

|Mot|Freq A|Rang A|Freq B|Rang B|Différence (absolue)|
|-|:-:|:-:|:-:|:-:|:-:|
|chat|12|1|85|1|0|
|chien|8|4|22|4|0|
|eau|9|2|17|5|3|
|lait|9|3|42|2|1|
|boire|6|5|38|3|2|

Remarquons l’égalité de rang dans le corpus A entre les mots *eau* et *lait* qui partagent la même mesure de fréquence et corrigeons le tableau :

|Mot|Freq A|Rang A|Freq B|Rang B|Différence (absolue)|
|-|:-:|:-:|:-:|:-:|:-:|
|chat|12|1|85|1|0|
|chien|8|4|22|4|0|
|eau|9|2,5|17|5|2,5|
|lait|9|2,5|42|2|0,5|
|boire|6|5|38|3|2|

Le coefficient de Spearman vaut ainsi :

$$
\begin{align}
    \rho &= 1 - \frac{6 \cdot (2,5^2 + 0,5^2)}{5 \cdot (5^2 -1)}\\
    &= 1 - \frac{6 \cdot 6,50}{5 \cdot 24}\\
    &= 1 - \frac{39}{120}\\
    &= 0,675
\end{align}
$$

## La mesure Delta de Burrows

Issue d’un papier ["‘Delta’: a Measure of Stylistic Difference and a Guide to Likely Authorship"](https://doi-org.inshs.bib.cnrs.fr/10.1093/llc/17.3.267) paru en 2002 dans la revue *Literary and Linguistic Computing*, la mesure utilise les fréquences relatives des mots les plus courants dans des textes dont les auteurs ou les autrices sont connues afin d’établir une distance avec d’autres textes dont la paternité ou la maternité est disputée.

La méthode exposée consiste à :

1. créer un sous-corpus par auteur ou autrice qui sera constitué des textes pour lesquels la paternité ou la maternité n’est pas contestée ;
2. réunir tous ces textes dans un corpus général ;
3. déterminer les *n* mots les plus fréquents de ce corpus ;
4. pour chacun de ces *n* mots, calculer sa probabilité d’apparition dans chacun des sous-corpus ;
5. établir la moyenne et l’écart-type puis en déduire la valeur-z ;
6. répéter les opérations pour les textes anonymes ou dont l’attribution est contestée ;
7. calculer la cote $\Delta$.

Pour rappel, la formule de la valeur-z est identique à la variable centrée-réduite :

$$
Z = \frac{X-\mu}{\sigma}
$$

Pour la mesure du $\Delta$ de Burrows, utiliser la distance de Manhattan :

$$
\Delta (C, T) = \sum_{i=1}^n \lvert Z_i(C) - Z_i(T) \rvert
$$

### Pour aller plus loin

Suite à la proposition de John Burrows, des améliorations ont ensuite été proposées, comme le $\Delta$ quadratique ([Argamon, 2007](https://doi.org/10.1093/llc/fqn003)) ou encore le $\Delta \cos$ ([Smith and Aldridge, 2011](https://doi.org/10.1080/09296174.2011.533591)) qui calcule la similarité cosinus à partir des valeurs-z afin d’obtenir une mesure angulaire.